# Calculate Political Opinion Models

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.level = logging.INFO

from os import path
from corputil import FileCorpus
from corputil.utils import load_stopwords
from gensim.models.word2vec import LineSentence, Word2Vec

stopwords = load_stopwords(path.join('data', 'german.txt'))

## Training the Base Model

Calculate the base model (from german wiki), that is later used as a base for training the classification models.

In [ ]:
# sentences = LineSentence(path.join('data', 'Archive', 'Corpus_Wiki.txt'))
# base = Word2Vec(sentences, workers=4, size=300, window=10, min_count=5)

Save model to disk. Don't finalize the model because we need to train it with new data later!

In [ ]:
# base.save(path.join('models', 'word2vec', 'base.w2v'))
# base = None

## Training the Classifier

Train model for Die Linke.
Model is finalized to save RAM.

In [ ]:
file = path.join('data', 'Politics', 'Corpus_Linke_Fraktion.txt')
corpus = list(FileCorpus(file).sentences_token(stopwords=stopwords))

linke = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
linke.train(corpus, total_examples=len(corpus))
# linke.init_sims(replace=True) Doesn't work for now!
linke.save(path.join('models', 'word2vec', 'Linke.w2v'))
linke = None

Train model for SPD.
Model is finalized to save RAM.

In [ ]:
file = path.join('data', 'Politics', 'Corpus_SPD_Fraktion.txt')
corpus = list(FileCorpus(file).sentences_token(stopwords=stopwords))

spd = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
spd.train(corpus, total_examples=len(corpus))
# spd.init_sims(replace=True) Doesn't work for now!
spd.save(path.join('models', 'word2vec', 'SPD.w2v'))
spd = None

Train model for Die Grünen.
Model is finalized to save RAM.

In [ ]:
file = path.join('data', 'Politics', 'Corpus_Grüne_Fraktion.txt')
corpus = list(FileCorpus(file).sentences_token(stopwords=stopwords))

gruene = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
gruene.train(corpus, total_examples=len(corpus))
# gruene.init_sims(replace=True) Doesn't work for now!
gruene.save(path.join('models', 'word2vec', 'Grüne.w2v'))
gruene = None

Train model for FDP. Model is finalized to save RAM.

In [ ]:
file = path.join('data', 'Politics', 'Corpus_FDP_Fraktion.txt')
corpus = list(FileCorpus(file).sentences_token(stopwords=stopwords))

fdp = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
fdp.train(corpus, total_examples=len(corpus))
# fdp.init_sims(replace=True) Doesn't work for now!
fdp.save(path.join('models', 'word2vec', 'FDP.w2v'))
fdp = None

Train model for CDU. Model is finalized to save RAM.

In [ ]:
file = path.join('data', 'Politics', 'Corpus_CDU_Fraktion.txt')
corpus = list(FileCorpus(file).sentences_token(stopwords=stopwords))

cdu = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
cdu.train(corpus, total_examples=len(corpus))
# cdu.init_sims(replace=True) Doesn't work for now!
cdu.save(path.join('models', 'word2vec', 'CDU.w2v'))
cdu = None

Train model for NPD. Model is finalized to save RAM.

In [ ]:
file1 = path.join('data', 'Politics', 'Corpus_NPD_MV.txt')
file2 = path.join('data', 'Politics', 'Corpus_NPD_Sachsen.txt')
corpus = list(FileCorpus(file1, file2).sentences_token(stopwords=stopwords))

npd = Word2Vec.load(path.join('models', 'word2vec', 'base.w2v'))
npd.train(corpus, total_examples=len(corpus))
# npd.init_sims(replace=True) Doesn't work for now!
npd.save(path.join('models', 'word2vec', 'NPD.w2v'))
npd = None

## Classification

Load models and documents into memory.

In [ ]:
models = [path.join('models', 'word2vec', 'SPD.w2v'),
          path.join('models', 'word2vec', 'Linke.w2v'),
          path.join('models', 'word2vec', 'Grüne.w2v'),
          path.join('models', 'word2vec', 'FDP.w2v'),
          path.join('models', 'word2vec', 'CDU.w2v'),
          path.join('models', 'word2vec', 'NPD.w2v')]

In [ ]:
import pandas as pd
import numpy as np


def calc_score(doc, mod):
    model = Word2Vec.load(mod)
    score = model.score(doc, len(doc))
    return score

def calc_probability(docs, mods):
    sentlist = [s for d in docs for s in d]
    llhd = np.array( [ calc_score(sentlist, m) for m in mods ] )
    lhd = np.exp(llhd - llhd.max(axis=0))
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    prob = prob.sum()
    return prob/prob.sum()

# KW44 = path.join('data', 'CurrentNews', '2015KW44_Spiegel.txt')
# KW45 = path.join('data', 'CurrentNews', '2015KW45_Spiegel.txt')
# KW46 = path.join('data', 'CurrentNews', '2015KW46_Spiegel.txt')
# KW47 = path.join('data', 'CurrentNews', '2015KW47_Spiegel.txt')
# KW48 = path.join('data', 'CurrentNews', '2015KW48_Spiegel.txt')
# KW49 = path.join('data', 'CurrentNews', '2015KW49_Spiegel.txt')
# KW50 = path.join('data', 'CurrentNews', '2015KW50_Spiegel.txt')
# corpus = list(FileCorpus(KW44, KW45, KW46, KW47, KW48, KW49, KW50).doc_sentences_token(stopwords=stopwords))

file = path.join('data', 'CurrentNews', '2015KW44_Spiegel.txt')
corpus = list(FileCorpus(file).doc_sentences_token())
print(calc_probability(corpus, models))

Trained without stopwords, query without stopwords

Spiegel

Welt

Handelsblatt
0    0.091655
1    0.265744
2    0.192597
3    0.166589
4    0.197317
5    0.086098

Junge Freiheit

Junge Welt


Trained without stopwords, query with stopwords

Spiegel
0    0.093173
1    0.188099
2    0.164464
3    0.138946
4    0.159299
5    0.256019

Welt
0    0.087746
1    0.248905
2    0.142936
3    0.194455
4    0.130204
5    0.195753

Handelsblatt
0    0.083737
1    0.255106
2    0.185700
3    0.163751
4    0.148524
5    0.163182

Junge Freiheit
0    0.091618
1    0.213073
2    0.110979
3    0.201321
4    0.092264
5    0.290745

Junge Welt
0    0.045699
1    0.215526
2    0.091375
3    0.421882
4    0.076452
5    0.149066

Trained with stopwords, query without stopwords

Spiegel

Welt

Handelsblatt

Junge Freiheit

Junge Welt


Trained with stopwords, query with stopwords

Spiegel

Welt

Handelsblatt

Junge Freiheit

Junge Welt
